In [1]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, CategoricalArrays
using Plots
using Distributions

In [2]:
const GRB_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-11


In [256]:
df = CSV.read("imputed_table.csv", DataFrame)[:, 2:end];

people = CSV.read("people.csv", DataFrame);

In [202]:
RHO = 10;

In [58]:
# preferences_0 = CSV.read("preferences_0.csv", DataFrame);
# preferences_1 = CSV.read("preferences_1.csv", DataFrame);

# preferences_0 = Matrix(preferences_0)[:,2:end];
# preferences_1 = Matrix(preferences_1)[:,2:end];

In [266]:
preference_df = Matrix(df[193:232, 193:232]);

people_df = people[193:232, :];

In [270]:
df[1:20, 1:20]

,1.0,2.0,3.0,4.0,5.0,6.0,7.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.5,-0.132105,0.640423,0.1049,-0.535669,0.361595,1.304
2,-0.504199,-0.5,0.414866,-1.24978,0.175539,-0.320363,-1.90409
3,0.937358,0.21877,-0.5,1.39214,0.31772,0.545022,0.99023
4,0.135331,0.261963,0.7203,-0.5,-1.29184,-1.6005,0.553141
5,-1.23944,0.00437185,0.335725,1.19403,-0.5,-0.396796,-0.877103
6,-0.613547,1.13328,-0.784445,0.119175,-0.284907,-0.5,-0.559545
7,0.450715,-0.226007,0.66004,-1.7661,-0.100062,0.861031,-0.5
8,-0.489866,0.541139,-0.336873,0.770053,0.983356,1.98303,-0.412383
9,-0.234397,1.84195,0.398938,-1.88541,-1.23653,1.51647,-0.0490567


In [205]:
preference_df[diagind(preference_df)] .= RHO;

In [187]:
1:size([:agnostic, :atheist, :buddhist, :catholic, :hindu, :jewish, :mormon, :muslim, :protestant, :unaffiliated], 1)

1:10

In [193]:
size(religion_df)[0]

(40, 10)

In [273]:

RHO = 0.5
m1 = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables
@variable(m1, z[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)
@variable(m1, t[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)
@variable(m1, g[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)

# Create objective
@objective(m1, Min, -1 * sum(z[i,j] * (preference_df[i,j] + preference_df[j,i]) for i in 1:size(preference_df)[1], j in i:size(preference_df)[2]) + (size(preference_df)[1] - sum(z)) * RHO)

# Create constraints

# Each person can only be matched with one person
@constraint(m1, [i=1:size(preference_df)[1]], sum(z[i,j] for j in 1:size(preference_df)[2]) <= 1)
@constraint(m1, [j=1:size(preference_df)[2]], sum(z[i,j] for i in 1:size(preference_df)[1]) <= 1)

# Make sure that if i is matched with j, then j is matched with i
# Different gender matching
# People with strong religious preferences are matched with someone sharing the same religious preference
religion_df = Matrix(people_df[:, [:agnostic, :atheist, :buddhist, :catholic, :hindu, :jewish, :mormon, :muslim, :protestant, :unaffiliated]])

for i in 1:size(z, 1)
    for j in i:size(z, 2)
        @constraint(m1, z[i,j] == z[j,i])
        @constraint(m1, z[i,j] <= g[i,j])
        @constraint(m1, g[i,j] >= people_df[i, :gender] - people_df[j, :gender])
        @constraint(m1, -g[i,j] >= people_df[i, :gender] - people_df[j, :gender])
        # @constraint(m1, s[i,j] <= 1-(j-i))

        @constraint(m1, (1 - sum(religion_df[i, k] * religion_df[j, k] for k in 1:size(religion_df)[2])) * t[i,j] * z[i,j] <= 0)
        @constraint(m1, t[i,j] >= people_df[i, :imprelig])
        @constraint(m1, t[i,j] >= people_df[j, :imprelig])
    end
end

optimize!(m1)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5000 rows, 4800 columns and 9680 nonzeros
Model fingerprint: 0xae6ef205
Model has 820 quadratic constraints
Variable types: 0 continuous, 4800 integer (4800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e-04, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 20.0000000
Presolve removed 4961 rows and 4452 columns
Presolve time: 0.01s
Presolved: 315 rows, 486 columns, 972 nonzeros
Variable types: 0 continuous, 486 integer (486 binary)

Root relaxation: objective -3.224182e+01, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     

In [246]:
zij = value.(z);

sum(zij, dims=1)

1×40 Matrix{Float64}:
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  0.0  1.0

In [247]:
CSV.write("z.csv", Tables.table(zij))

"z.csv"

### Compared to the Heuristic Method, we get an objective value of 32.24 vs. 14.15, a 2.3x improvement!

Now let us try it out on the larger instance:



In [289]:
preference_df = Matrix(df[1:22, 1:22]);

people_df = people;

In [290]:
RHO = 0.5
m1 = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables
@variable(m1, z[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)
@variable(m1, t[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)
@variable(m1, g[1:size(preference_df, 1), 1:size(preference_df, 2)], Bin)

# Create objective
@objective(m1, Min, -1 * sum(z[i,j] * (preference_df[i,j] + preference_df[j,i]) for i in 1:size(preference_df)[1], j in i:size(preference_df)[2]) + (size(preference_df)[1] - sum(z)) * RHO)

# Create constraints

# Each person can only be matched with one person
@constraint(m1, [i=1:size(preference_df)[1]], sum(z[i,j] for j in 1:size(preference_df)[2]) <= 1)
@constraint(m1, [j=1:size(preference_df)[2]], sum(z[i,j] for i in 1:size(preference_df)[1]) <= 1)

# Make sure that if i is matched with j, then j is matched with i
# Different gender matching
# People with strong religious preferences are matched with someone sharing the same religious preference
religion_df = Matrix(people_df[:, [:agnostic, :atheist, :buddhist, :catholic, :hindu, :jewish, :mormon, :muslim, :protestant, :unaffiliated]])

for i in 1:size(z, 1)
    for j in i:size(z, 2)
        @constraint(m1, z[i,j] == z[j,i])
        @constraint(m1, z[i,j] <= g[i,j])
        @constraint(m1, g[i,j] >= people_df[i, :gender] - people_df[j, :gender])
        @constraint(m1, -g[i,j] >= people_df[i, :gender] - people_df[j, :gender])
        # @constraint(m1, s[i,j] <= 1-(j-i))

        @constraint(m1, (1 - sum(religion_df[i, k] * religion_df[j, k] for k in 1:size(religion_df)[2])) * t[i,j] * z[i,j] <= 0)
        @constraint(m1, t[i,j] >= people_df[i, :imprelig])
        @constraint(m1, t[i,j] >= people_df[j, :imprelig])
    end
end

optimize!(m1)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1562 rows, 1452 columns and 2948 nonzeros
Model fingerprint: 0x561b8502
Model has 253 quadratic constraints
Variable types: 0 continuous, 1452 integer (1452 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [4e-03, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1013 rows and 737 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

User-callback calls 32, time in user-callback 0.00 sec


In [77]:
rho = 0.5;
# Create a new model

m = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables

#zij = 1 if i and j are matched, 0 otherwise
@variable(m, z[1:size(preferences_0,1), 1:size(preferences_1,1)], Bin)
@variable(m, ui[1:size(preferences_0,1)], Bin)
@variable(m, uj[1:size(preferences_1,1)], Bin)

# Counting variable for unmatched students

#t represents max of mi and mj
#@variable(m, t[1:size(preferences_0,1), 1:size(preferences_1,1)], Bin)

# Create objective

@objective(m, Min, -1*sum(z[i,:]'*(preferences_0[i,:] + preferences_1'[i,:]) for i in 1:size(preferences_0,1)) + rho*(sum(ui) + sum(uj)))

# Create constraints

# Each person can only be matched to one person 
@constraint(m, [i in 1:size(preferences_0,1)], sum(z[i,j] for j in 1:size(preferences_1,1)) <= 1)
@constraint(m, [j in 1:size(preferences_1,1)], sum(z[i,j] for i in 1:size(preferences_0,1)) <= 1)

# If unmatched define ui and uj
@constraint(m, [i in 1:size(preferences_0,1)], ui[i] >= 1 - sum(z[i,j] for j in 1:size(preferences_1,1)))
@constraint(m, [j in 1:size(preferences_1,1)], uj[j] >= 1 - sum(z[i,j] for i in 1:size(preferences_0,1)))

optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 440 columns and 1640 nonzeros
Model fingerprint: 0x00792b20
Variable types: 0 continuous, 440 integer (440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -20.4384452
Presolve removed 0 rows and 92 columns
Presolve time: 0.00s
Presolved: 80 rows, 348 columns, 1272 nonzeros
Variable types: 0 continuous, 348 integer (348 binary)

Root relaxation: objective -4.347703e+01, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     -43.4770278  -43.47703  0.00%     -    0s

Explored 1 nodes (28 simplex iterati

In [79]:
# Get value of zij

zij = value.(z)

20×20 Matrix{Float64}:
  0.0  -0.0   0.0  -0.0   0.0   0.0  …   0.0  -0.0   0.0  -0.0   0.0  -0.0
 -0.0   0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0   0.0   1.0   0.0   0.0
 -0.0  -0.0  -0.0   0.0   0.0   1.0     -0.0   0.0  -0.0  -0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0   0.0  -0.0     -0.0   0.0  -0.0   0.0   1.0   0.0
  0.0  -0.0  -0.0   0.0   0.0   0.0      1.0  -0.0  -0.0   0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0  …   0.0   0.0   0.0  -0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0      0.0   0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   0.0  -0.0   0.0  -0.0     -0.0  -0.0   1.0  -0.0  -0.0  -0.0
  1.0  -0.0  -0.0  -0.0  -0.0  -0.0      0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0   1.0
 -0.0  -0.0   1.0  -0.0  -0.0  -0.0  …   0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0  -0.0   0.0  -0.0   0.0     -0.0  -0.0  -0.0  -0.0   0.0  -0.0
 -

In [89]:
sum((preferences_0 + preferences_1') .* zij)

20.0